In [7]:
from scipy import signal
import cv2
import sys
import glob
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import os
from PIL import Image
import xlrd
import math
from pylab import *
from scipy import signal
import numpy as np
from matplotlib import pyplot as plt
from skimage.feature import greycomatrix, greycoprops
from skimage import data

In [8]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(10,10))

In [9]:
wb = xlrd.open_workbook(os.getcwd() + "/Drishti-GS1_files/Drishti-GS1_diagnosis.xlsx")
sheet = wb.sheet_by_index(0)
val = [sheet.col_values(1)[5:], sheet.col_values(8)[5:]]

In [10]:
def load_image(path):
    return np.asarray(Image.open(path))

In [11]:
def load_set(folder, shuffle=False):
    img_list = sorted(glob.glob(os.path.join(folder, '*.png')) + \
                      glob.glob(os.path.join(folder, '*.jpg')) + \
                      glob.glob(os.path.join(folder, '*.jpeg')))
    if shuffle:
        np.random.shuffle(img_list)
    data = []
    filenames = []
    for img_fn in img_list:
        img = load_image(img_fn)
        data.append(img)
        filenames.append(img_fn)
    return data, filenames

In [12]:
def extract_DRISHTI_GS_train(db_folder, cdr, train_data):
    file_codes_all, exp1, exp2, exp3, exp4 = [], [], [], [], []
    if train_data:
        set_path = os.path.join(db_folder, 'Drishti-GS1_files', 'Drishti-GS1_files', 'Training')
    else:
        set_path = os.path.join(db_folder, 'Drishti-GS1_files', 'Drishti-GS1_files', 'Test')
    images_path = os.path.join(set_path, 'Images')
    X_all, file_names = load_set(images_path)
    rel_file_names = [os.path.split(fn)[-1] for fn in file_names]
    rel_file_names_wo_ext = [fn[:fn.rfind('.')] for fn in rel_file_names]
    if train_data:
        file_codes = [fn[fn.find('_'):] for fn in rel_file_names_wo_ext]
    else:
        file_codes = [fn[fn.find('_'):] for fn in rel_file_names_wo_ext]
    file_codes_all.extend(file_codes)

    for fn in rel_file_names_wo_ext:
        if cdr:
            if train_data:
                CDR = open(os.path.join(set_path, 'GT', fn, fn + '_cdrValues.txt'), 'r')
            else:
                CDR = open(os.path.join(set_path, 'Test_GT', fn, fn + '_cdrValues.txt'), 'r')
            CDR = list(CDR)
            CDR = CDR[0].split()
            exp1.append(CDR[0])
            exp2.append(CDR[1])
            exp3.append(CDR[2])
            exp4.append(CDR[3])

    return X_all, file_codes_all, exp1, exp2, exp3, exp4, file_names

In [13]:
X_all, file_codes_all, exp1, exp2, exp3, exp4, file_names = extract_DRISHTI_GS_train(os.getcwd(), True, False)

In [14]:
X_all

[array([[[3, 1, 3],
         [3, 1, 3],
         [4, 2, 4],
         ...,
         [2, 0, 2],
         [3, 1, 3],
         [4, 2, 4]],
 
        [[2, 0, 2],
         [3, 1, 3],
         [4, 2, 4],
         ...,
         [2, 0, 2],
         [3, 1, 3],
         [4, 2, 4]],
 
        [[3, 1, 3],
         [4, 2, 4],
         [4, 2, 4],
         ...,
         [4, 2, 4],
         [4, 2, 4],
         [4, 2, 4]],
 
        ...,
 
        [[5, 2, 4],
         [4, 1, 3],
         [4, 1, 3],
         ...,
         [3, 1, 3],
         [3, 1, 3],
         [3, 1, 3]],
 
        [[5, 2, 4],
         [4, 1, 3],
         [4, 1, 3],
         ...,
         [3, 1, 3],
         [3, 1, 3],
         [3, 1, 3]],
 
        [[5, 2, 4],
         [4, 1, 3],
         [4, 1, 3],
         ...,
         [5, 2, 4],
         [5, 2, 4],
         [3, 1, 3]]], dtype=uint8),
 array([[[4, 1, 3],
         [3, 0, 2],
         [4, 2, 4],
         ...,
         [2, 0, 1],
         [3, 0, 2],
         [3, 1, 3]],
 
        [[1, 

In [15]:
def segment(image, plot_seg, plot_hist):
    image = image[400:1400, 500:1600, :]  # cropping the fundus image to ger region of interest

    Abo, Ago, Aro = cv2.split(image)  # splitting into 3 channels
    # Aro = clahe.apply(Aro)
    Ago = clahe.apply(Ago)
    M = 60  # filter size
    filter = signal.gaussian(M, std=6)  # Gaussian Window
    filter = filter / sum(filter)
    STDf = filter.std()  # It'standard deviation

    Ar = Aro - Aro.mean() - Aro.std()  # Preprocessing Red

    Mr = Ar.mean()  # Mean of preprocessed red
    SDr = Ar.std()  # SD of preprocessed red
    Thr = 0.5 * M - STDf - Ar.std()  # Optic disc Threshold
    # print(Thr)

    Ag = Ago - Ago.mean() - Ago.std()  # Preprocessing Green
    Mg = Ag.mean()  # Mean of preprocessed green
    SDg = Ag.std()  # SD of preprocessed green
    Thg = 0.5 * Mg + 2 * STDf + 2 * SDg + Mg  # Optic Cup Threshold
    # print(Thg)

    hist, bins = np.histogram(Ag.ravel(), 256, [0, 256])  # Histogram of preprocessed green channel
    histr, binsr = np.histogram(Ar.ravel(), 256, [0, 256])  # Histogram of preprocessed red channel

    smooth_hist_g = np.convolve(filter, hist)  # Histogram Smoothing Green
    smooth_hist_r = np.convolve(filter, histr)  # Histogram Smoothing Red

    # plot histogram if input is true
    if plot_hist:
        plt.subplot(2, 2, 1)
        plt.plot(hist)
        plt.title("Preprocessed Green Channel")

        plt.subplot(2, 2, 2)
        plt.plot(smooth_hist_g)
        plt.title("Smoothed Histogram Green Channel")

        plt.subplot(2, 2, 3)
        plt.plot(histr)
        plt.title("Preprocessed Red Channel")

        plt.subplot(2, 2, 4)
        plt.plot(smooth_hist_r)
        plt.title("Smoothed Histogram Red Channel")

        plt.show()

    r, c = Ag.shape
    Dd = np.zeros(shape=(r, c))  # Segmented disc image initialization
    Dc = np.zeros(shape=(r, c))  # Segmented cup image initialization

    # Using obtained threshold for thresholding of the fundus image
    for i in range(1, r):
        for j in range(1, c):
            if Ar[i, j] > Thr:
                Dd[i, j] = 255
            else:
                Dd[i, j] = 0

    for i in range(1, r):
        for j in range(1, c):

            if Ag[i, j] > Thg:
                Dc[i, j] = 1
            else:
                Dc[i, j] = 0

    # Saving the segmented image in the same place as the code folder
    cv2.imwrite('disk.png', Dd)
    plt.imsave('cup.png', Dc)

    if plot_seg:
        plt.imshow(Dd, cmap='gray', interpolation='bicubic')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.show()

        plt.imshow(Dc, cmap='gray', interpolation='bicubic')
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()

In [17]:
def cdr(cup, disc, plot):
    # morphological closing and opening operations
    R1 = cv2.morphologyEx(cup, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)), iterations=1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)), iterations=1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 21)), iterations=1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (21, 1)), iterations=1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (33, 33)), iterations=1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (43, 43)), iterations=1)

    img = clahe.apply(r3)

    ret, thresh = cv2.threshold(cup, 127, 255, 0)
    # Getting all possible contours in the segmented image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cup_diameter = 0
    largest_area = 0
    el_cup = contours[0]
    if len(contours) != 0:
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i])  # Getting the contour with the largest area
                if (area > largest_area):
                    largest_area = area
                    index = i
                    el_cup = cv2.fitEllipse(contours[i])

    cv2.ellipse(img, el_cup, (140, 60, 150), 3)  # fitting ellipse with the largest area
    x, y, w, h = cv2.boundingRect(contours[index])  # fitting a rectangle on the ellipse to get the length of major axis
    cup_diameter = max(w, h)  # major axis is the diameter

    # morphological closing and opening operations
    R1 = cv2.morphologyEx(disc, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)), iterations=1)
    r1 = cv2.morphologyEx(R1, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)), iterations=1)
    R2 = cv2.morphologyEx(r1, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 21)), iterations=1)
    r2 = cv2.morphologyEx(R2, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (21, 1)), iterations=1)
    R3 = cv2.morphologyEx(r2, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (33, 33)), iterations=1)
    r3 = cv2.morphologyEx(R3, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (43, 43)), iterations=1)

    img2 = clahe.apply(r3)

    ret, thresh = cv2.threshold(disc, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL,
                                                 cv2.CHAIN_APPROX_SIMPLE)  # Getting all possible contours in the segmented image
    disk_diameter = 0
    largest_area = 0
    el_disc = el_cup
    if len(contours) != 0:
        for i in range(len(contours)):
            if len(contours[i]) >= 5:
                area = cv2.contourArea(contours[i])  # Getting the contour with the largest area
                if (area > largest_area):
                    largest_area = area
                    index = i
                    el_disc = cv2.fitEllipse(contours[i])

    cv2.ellipse(img2, el_disc, (140, 60, 150), 3)  # fitting ellipse with the largest area
    x, y, w, h = cv2.boundingRect(contours[index])  # fitting a rectangle on the ellipse to get the length of major axis
    disk_diameter = max(w, h)  # major axis is the diameter

    if plot:
        plt.imshow(img2, cmap='gray', interpolation='bicubic')
        plt.axis("off")
        plt.title("Optic Disk")
        plt.show()
        plt.imshow(img)
        plt.axis("off")
        plt.title("Optic Cup")
        plt.show()

    if (disk_diameter == 0): return 1  # if disc not segmented properly then cdr might be infinity
    cdr = cup_diameter / disk_diameter  # ration of major axis of cup and disc
    return cdr

In [18]:
CDR = []  # load calculated cdr here
VAL = []  # load their labels here
count = 0
for i in range(len(X_all)):
    set_path = os.path.join(os.getcwd(), 'Drishti-GS1_files', 'Drishti-GS1_files', 'Test',
                            file_names[i])  # folder to the test image in dataset(don't change file_names[])
    image = cv2.imread(set_path, 1)
    segment(image, False, False)
    # images will be saved in the same folder as the code so that folder needs to be put here
    cup = cv2.imread(os.getcwd()+'/cup.png', 0)
    # images will be saved in the same folder as the code so that folder needs to be put here
    disc = cv2.imread(os.getcwd()+ '/disk.png', 0)
    cdr_cal = cdr(cup, disc, False)
    if (val[1][int(file_codes_all[count][1:]) - 1] == 'Glaucomatous'):
        VAL.append(1)
    else:
        VAL.append(0)
    CDR.append(cdr_cal)
    print(file_codes_all[count], 'Exp1_cdr:', exp1[count], 'Exp2_cdr:', exp2[count], 'Exp3_cdr:', exp3[count],
          'Exp4_cdr:', exp4[count], 'Pred_cdr:', cdr_cal)
    # put same folder as that of cup and disc so that they can be removed for processing of next set
    os.remove(os.getcwd() + '/cup.png')
    os.remove(os.getcwd() + '/disk.png')
    count += 1

error1, error2, error3, error4 = [], [], [], []
# calculated error against each expert
for i in range(len(X_all)):
    error1.append(float(exp1[i]) - CDR[i])
    error2.append(float(exp2[i]) - CDR[i])
    error3.append(float(exp3[i]) - CDR[i])
    error4.append(float(exp4[i]) - CDR[i])

_001 Exp1_cdr: 0.85 Exp2_cdr: 0.82 Exp3_cdr: 0.80 Exp4_cdr: 0.82 Pred_cdr: 0.7160751565762005
_003 Exp1_cdr: 0.83 Exp2_cdr: 0.79 Exp3_cdr: 0.72 Exp4_cdr: 0.79 Pred_cdr: 0.7281323877068558
_005 Exp1_cdr: 0.86 Exp2_cdr: 0.87 Exp3_cdr: 0.81 Exp4_cdr: 0.80 Pred_cdr: 0.7216312056737588
_006 Exp1_cdr: 0.64 Exp2_cdr: 0.77 Exp3_cdr: 0.65 Exp4_cdr: 0.53 Pred_cdr: 0.4430992736077482
_007 Exp1_cdr: 0.86 Exp2_cdr: 0.83 Exp3_cdr: 0.70 Exp4_cdr: 0.62 Pred_cdr: 0.6280087527352297
_009 Exp1_cdr: 0.54 Exp2_cdr: 0.56 Exp3_cdr: 0.39 Exp4_cdr: 0.53 Pred_cdr: 0.9064171122994652
_011 Exp1_cdr: 0.81 Exp2_cdr: 0.81 Exp3_cdr: 0.77 Exp4_cdr: 0.83 Pred_cdr: 0.9510086455331412
_013 Exp1_cdr: 0.80 Exp2_cdr: 0.66 Exp3_cdr: 0.65 Exp4_cdr: 0.62 Pred_cdr: 0.39879759519038077
_014 Exp1_cdr: 0.86 Exp2_cdr: 0.87 Exp3_cdr: 0.84 Exp4_cdr: 0.82 Pred_cdr: 0.9850374064837906
_019 Exp1_cdr: 0.86 Exp2_cdr: 0.91 Exp3_cdr: 0.84 Exp4_cdr: 0.78 Pred_cdr: 0.593939393939394
_020 Exp1_cdr: 0.88 Exp2_cdr: 0.90 Exp3_cdr: 0.80 Exp4_cdr: 